**Import required libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Lambda, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

**Load and reshape data**

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

X_all = (train.ix[:,1:].values).astype('float32')
X_all = X_all.reshape(X_all.shape[0], 28, 28,1)
X_all = X_all / 255.0 # normalize

Y_all = train.ix[:,0].values.astype('int32') 
Y_all= to_categorical(Y_all) # One Hot encoding of labels

X_train, X_validate, Y_train, Y_validate = train_test_split(X_all, Y_all, test_size = 0.2, random_state=123)

X_test = test.values.astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test = X_test / 255.0 # normalize

**A simple model**

In [ ]:
model= Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(28, 28, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.20))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.005),
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy'])

**Learn**

In [ ]:
batch_size = 64
epochs = 20
input_shape = (28, 28, 1)

h = model.fit(
        X_train,
        Y_train, 
        batch_size=batch_size,
        epochs = epochs, 
        validation_data = (X_validate, Y_validate),
        verbose = 1)

final_loss, final_acc = model.evaluate(X_validate, Y_validate, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

**Plot history**

In [ ]:
loss = h.history['loss']
val_loss = h.history['val_loss']
epochs = range(len(loss))
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

**Load and reshape test data**

In [ ]:
predicted_classes = model.predict_classes(X_test)

submissions = pd.DataFrame({"ImageId": list(range(1, len(predicted_classes) + 1)), "Label": predicted_classes})
submissions.to_csv("submission.csv", index=False, header=True)